In [80]:
import os
import numpy as np
import cv2
import pandas as pd
import k3d
import open3d as o3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, clear_output
import ipywidgets as widgets

# ---------- KITTI Utilities ---------- #

# Load LIDAR point cloud from a .bin file
def load_lidar_bin(file_path):
    # Load binary data and reshape to N x 4 (x, y, z, reflectance)
    points = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
    # Return only x, y, z (ignore reflectance)
    return points[:, :3]

# Extract object detection labels from KITTI label file
def extract_labels(file_path):
    columns = [
        "ObjectType", "Truncation", "Occlusion", "Alpha", 
        "X1", "Y1", "X2", "Y2", "H", "W", "L", "X", "Y", "Z", "Rotation_Y"
    ]
    data = []
    with open(file_path, "r") as file:
        for line in file:
            values = line.strip().split()
            # Create a dictionary for each object, converting all values to float except ObjectType
            object_data = {
                columns[i]: values[i] if i == 0 else float(values[i])
                for i in range(len(columns))
            }
            data.append(object_data)
    # Return as a Pandas DataFrame
    return pd.DataFrame(data)

# Extract object detection labels from KITTI label file
def extract_predictions(file_path):
    columns = [
        "ObjectType", "Truncation", "Occlusion", "Alpha", 
        "X1", "Y1", "X2", "Y2", "L", "H", "W", "X", "Y", "Z", "Rotation_Y"
    ]
    data = []
    with open(file_path, "r") as file:
        for line in file:
            values = line.strip().split()
            # Create a dictionary for each object, converting all values to float except ObjectType
            object_data = {
                columns[i]: values[i] if i == 0 else float(values[i])
                for i in range(len(columns))
            }
            data.append(object_data)
    # Return as a Pandas DataFrame
    return pd.DataFrame(data)

# Load calibration matrices (projection, rectification, and transformation) from file
def extract_matrices(filename):
    global P2, P3, R0_rect, Tr_velo_to_cam, Tr_cam_to_velo
    with open(filename, 'r') as file:
        for line in file:
            if line.strip():
                parts = line.split(':')
                key = parts[0].strip()
                values = np.fromstring(parts[1], sep=' ')
                # Parse and reshape the appropriate matrix
                if key == "P2":
                    P2 = values.reshape(3, 4)
                elif key == "P3":
                    P3 = values.reshape(3, 4)
                elif key == "R0_rect":
                    R0_rect = values.reshape(3, 3)
                elif key == "Tr_velo_to_cam":
                    matrix_3x4 = values.reshape(3, 4)
                    # Extend to 4x4 for homogeneous transformation
                    Tr_velo_to_cam = np.vstack([matrix_3x4, np.array([[0, 0, 0, 1]])])
                    # Inverse to get camera to lidar transformation
                    Tr_cam_to_velo = np.linalg.inv(Tr_velo_to_cam)

def compute_3D_from_disparity(right_proj_mat, disparity_img, R0_rect=None):
    """
    Compute a 3D point cloud from a disparity image using the stereo projection matrix.

    Parameters:
    - right_proj_mat: 3x4 right projection matrix 
    - disparity_img: 2D array of disparity values
    - R0_rect (optional): 3x3 rectification matrix to convert to rectified camera coordinates

    Returns:
    - xyz_img: (H, W, 3) array of 3D points in camera (or rectified) coordinates
    """

    # Convert disparity from uint16 to float if needed (KITTI format stores disparity * 256)
    if disparity_img.dtype == np.uint16:
        disparity_img = disparity_img.astype(np.float32) / 256.0

    height, width = disparity_img.shape
    xyz_img = np.full((height, width, 3), np.nan, dtype=np.float32)  # Default to NaNs

    # Extract camera intrinsics
    f = right_proj_mat[0, 0]             # Focal length (assumed fx = fy)
    c_x = right_proj_mat[0, 2]           # Principal point x
    c_y = right_proj_mat[1, 2]           # Principal point y
    T_x = right_proj_mat[0, 3]           # Translation offset (fx * baseline)

    # Compute baseline from T_x and focal length
    baseline = -T_x / f            # baseline (in meters)

    # Create a mask for valid disparity values
    mask = disparity_img > 0
    disparity = disparity_img.copy()

    # Compute depth: Z = f * B / disparity
    depth = np.full_like(disparity, np.nan, dtype=np.float32)
    depth[mask] = f * baseline / disparity[mask]

    # Generate mesh grid of pixel coordinates (u, v)
    u, v = np.meshgrid(np.arange(width), np.arange(height))

    # Compute X, Y, Z in camera coordinates
    X = (u - c_x) * depth / f
    Y = (v - c_y) * depth / f
    Z = depth

    # Stack into 3D points
    xyz_cam = np.stack((X, Y, Z), axis=-1)

    # Apply rectification if needed
    if R0_rect is not None:
        xyz_flat = xyz_cam.reshape(-1, 3)
        xyz_rect = R0_rect @ xyz_flat.T
        xyz_img = xyz_rect.reshape(height, width, 3)
    else:
        xyz_img = xyz_cam

    # # Optional: mask out distant points (e.g., beyond 17.5 meters)
    # xyz_img[xyz_img[:, :, 2] > 17.5] = np.nan
    print(xyz_img.shape)

    return xyz_img


def filter_lidar_fov(points, fov_degrees=120):
    """
    Filters 3D LIDAR points to keep only those within a given horizontal field of view (in front of the camera).
    
    Parameters:
    - points: Nx3 array of points in camera coordinates
    - fov_degrees: field of view angle in degrees (default: 120)

    Returns:
    - Filtered points within the field of view
    """
    # Compute horizontal angles (in degrees) relative to camera's forward axis
    x, y = points[:, 0], points[:, 1]
    angles = np.degrees(np.arctan2(y, x))  # Angle in degrees between -180 and 180

    # Define limits (centered around 0)
    half_fov = fov_degrees / 2.0
    mask = (angles >= -half_fov) & (angles <= half_fov)

    return points[mask]

def remove_ground_plane(points, height_threshold=-1.0, axis='z', direction='above'):
    """
    Removes ground points by filtering points below a certain height.

    Parameters:
    - points: Nx3 numpy array
    - height_threshold: height cutoff
    - axis: axis to use for height

    Returns:
    - filtered_points: points above the ground
    """
    axis_map = {'x': 0, 'y': 1, 'z': 2}
    axis_idx = axis_map.get(axis.lower(), 2)

    if direction == 'above':
        return points[points[:, axis_idx] > height_threshold]
    elif direction == 'below':
        return points[points[:, axis_idx] < height_threshold]
    else:
        raise ValueError("Direction must be 'above' or 'below'")

def clip_point_cloud(pcd, max_range=20.0):
    points = np.asarray(pcd.points)
    mask = np.linalg.norm(points, axis=1) < max_range
    pcd.points = o3d.utility.Vector3dVector(points[mask])
    return pcd

def compute_hausdorff_and_stats(pcd1, pcd2):
    """Compute symmetric Hausdorff and average distances."""
    d1 = np.asarray(pcd1.compute_point_cloud_distance(pcd2))
    d2 = np.asarray(pcd2.compute_point_cloud_distance(pcd1))

    hausdorff = max(np.max(d1), np.max(d2))
    mean1, mean2 = np.mean(d1), np.mean(d2)
    percentile_95 = np.percentile(np.concatenate([d1, d2]), 95)

    return hausdorff, mean1, mean2, percentile_95

# Transform 3D LIDAR points from LIDAR to camera coordinate system
def transform_lidar_to_camera(lidar_points, Tr_velo_to_cam):

    # Convert to homogeneous coordinates by adding 1
    ones = np.ones((lidar_points.shape[0], 1))
    lidar_hom = np.hstack((lidar_points, ones))

    # Apply transformation matrix
    cam_points = lidar_hom @ Tr_velo_to_cam.T

    return cam_points[:, :3]  # Return only x, y, z (discard homogeneous coordinate)

# Display stereo image pair side by side
def plot_img(left_path, right_path, figsize=(22, 6)):
    img_left = mpimg.imread(left_path)
    img_right = mpimg.imread(right_path)
    combined_img = np.hstack((img_left, img_right))
    plt.figure(figsize=figsize)
    plt.imshow(combined_img)
    plt.axis("off")
    plt.show()

def rgb_to_int(rgb_colors):
    """
    rgb_colors: np.array of size Nx3
    return: np.array of size N
    """
    # Avoid overflow issues with uint8.
    rgb_colors = rgb_colors.astype(np.uint32).T
    int_colors = (rgb_colors[0] << 16) + (rgb_colors[1] << 8) + rgb_colors[2]
    return int_colors

def bgr_to_int(bgr_colors):
    """
    bgr_colors: np.array of size Nx3
    return: np.array of size N
    """
    return rgb_to_int(bgr_colors[:, ::-1])

# def compute_box_corners(label_row):
#     # Dimensions
#     h, w, l = label_row['H'], label_row['W'], label_row['L']
#     x, y, z = label_row['X'], label_row['Y'], label_row['Z']
#     ry = label_row['Rotation_Y']
    
#     # Create a bounding box in object coordinates
#     corners = np.array([
#         [ l/2,  l/2, -l/2, -l/2,  l/2,  l/2, -l/2, -l/2],
#         [  0 ,   0 ,   0 ,   0 , -h , -h ,  -h ,  -h ],
#         [ w/2, -w/2, -w/2,  w/2,  w/2, -w/2, -w/2,  w/2]
#     ])

#     # Rotation around Y-axis
#     R = np.array([
#         [ np.cos(ry), 0, np.sin(ry)],
#         [ 0,          1, 0         ],
#         [-np.sin(ry), 0, np.cos(ry)]
#     ])
#     rotated = R @ corners
#     translated = rotated + np.array([[x], [y], [z]])
#     return translated.T  # (8, 3)

def filter_labels_in_fov(labels_df, R0_rect, fov_degrees=80):
    """
    Filters KITTI label objects so that at least one bounding box corner lies in the camera's horizontal FOV.

    Parameters:
        labels_df: KITTI labels dataframe
        R0_rect: 3x3 rectification matrix
        fov_degrees: horizontal FOV angle in degrees

    Returns:
        Filtered label dataframe
    """
    def compute_box_corners_camera_coords(row):
        h, w, l = row['H'], row['W'], row['L']
        x, y, z = row['X'], row['Y'], row['Z']
        ry = row['Rotation_Y']

        # 3D bounding box in local object frame (camera coordinates)
        x_corners = [ l/2,  l/2, -l/2, -l/2,  l/2,  l/2, -l/2, -l/2]
        y_corners = [  0 ,   0 ,   0 ,   0 , -h , -h ,  -h ,  -h ]
        z_corners = [ w/2, -w/2, -w/2,  w/2,  w/2, -w/2, -w/2,  w/2]

        corners = np.array([x_corners, y_corners, z_corners])
        
        # Rotation around Y-axis
        R = np.array([
            [ np.cos(ry), 0, np.sin(ry)],
            [ 0,          1, 0         ],
            [-np.sin(ry), 0, np.cos(ry)]
        ])
        corners_rot = R @ corners
        corners_3d = corners_rot + np.array([[x], [y], [z]])  # Translate
        return corners_3d.T  # shape (8, 3)

    def in_fov(points_cam, fov_degrees):
        x, z = points_cam[:, 0], points_cam[:, 2]
        angles = np.degrees(np.arctan2(x, z))  # Horizontal angle
        half_fov = fov_degrees / 2.0
        return np.any((angles >= -half_fov) & (angles <= half_fov) & (z > 0))

    filtered_rows = []
    for _, row in labels_df.iterrows():
        corners = compute_box_corners_camera_coords(row)
        corners_rect = (R0_rect @ corners.T).T
        if in_fov(corners_rect, fov_degrees):
            filtered_rows.append(row)

    return pd.DataFrame(filtered_rows)



def draw_3d_bounding_boxes_k3d_with_labels(plot, labels_df, R0_rect, color=0xff0000):
    def compute_box_corners_camera_coords(row):
        h, w, l = row['H'], row['W'], row['L']
        x, y, z = row['X'], row['Y'], row['Z']
        ry = row['Rotation_Y']

        x_corners = [ l/2,  l/2, -l/2, -l/2,  l/2,  l/2, -l/2, -l/2]
        y_corners = [  0 ,   0 ,   0 ,   0 , -h , -h ,  -h ,  -h ]
        z_corners = [ w/2, -w/2, -w/2,  w/2,  w/2, -w/2, -w/2,  w/2]

        corners = np.array([x_corners, y_corners, z_corners])
        
        R = np.array([
            [ np.cos(ry), 0, np.sin(ry)],
            [ 0,          1, 0         ],
            [-np.sin(ry), 0, np.cos(ry)]
        ])
        corners_rot = R @ corners
        corners_3d = corners_rot + np.array([[x], [y], [z]])
        return corners_3d.T

    def get_box_lines(corners):
        lines = [
            (0,1), (1,2), (2,3), (3,0),
            (4,5), (5,6), (6,7), (7,4),
            (0,4), (1,5), (2,6), (3,7)
        ]
        return [(corners[i], corners[j]) for i, j in lines]

    for _, row in labels_df.iterrows():
        corners = compute_box_corners_camera_coords(row)
        corners_rect = (R0_rect @ corners.T).T
        lines = get_box_lines(corners_rect)

        vertices = []
        indices = []

        for i, (start, end) in enumerate(lines):
            vertices.extend([start, end])
            indices.append([2*i, 2*i + 1])

        # Draw 3D box
        plot += k3d.line(
            np.array(vertices, dtype=np.float32),
            indices=np.array(indices),
            color=color,
            width=0.005
        )

        # Place label above the top face
        top_face_indices = [4, 5, 6, 7]  # top face corners
        top_center = np.mean(corners_rect[top_face_indices], axis=0)
        label_pos = top_center + np.array([0, 0.5, 0])  # Shift text above the box

        plot += k3d.text(
            row['ObjectType'],
            position=label_pos.astype(np.float32),
            color=color,
            size=1.0,                # Larger text
            label_box=False,
            is_html=False
        )



In [81]:
# file_path_dataset = os.path.expanduser("~/CV_Simina")
# lidar_relative_path = "/velodyne/{:06d}.bin"
# left_img_relative_path = "/image_2/{:06d}.png"
# right_img_relative_path = "/image_3/{:06d}.png"
# disparity_relative_path = "/disparity_images/disparity{:06d}.png"
# labels_relative_path = "/data_object_label_2/{:06d}.txt"
# calib_relative_path ="/data_object_calib/training/calib/{:06d}.txt"

file_path_dataset = os.path.expanduser("~/Computer_vision/kitti_lidar/testing")
file_path_dataset_test = os.path.expanduser("~/Computer_vision/Performance/Test_set")
lidar_relative_path = "/velodyne/{:06d}.bin"
left_img_relative_path = "/image_2/{:06d}.png"
right_img_relative_path = "/image_3/{:06d}.png"
disparity_relative_path = "/disparity_images/disparity{:06d}.png"
labels_relative_path = "/label_2/{:06d}.txt"
calib_relative_path ="//calib/{:06d}.txt"
predictions_lidar = "/lidar/submit/{:06d}.txt"

sequence_number = 924
paths = {"lidar": file_path_dataset + lidar_relative_path.format(sequence_number),
        "left_img": file_path_dataset + left_img_relative_path.format(sequence_number),
        "right_img": file_path_dataset + right_img_relative_path.format(sequence_number),
        "disparity": file_path_dataset + disparity_relative_path.format(sequence_number),
        "labels": file_path_dataset + labels_relative_path.format(sequence_number),
        "calib": file_path_dataset + calib_relative_path.format(sequence_number),
        "pred_lidar": file_path_dataset_test + predictions_lidar.format(sequence_number)}

In [ ]:
print(f"Frame: {sequence_number}")

# Load calibration matrices and transform LIDAR points to camera coordinates
extract_matrices(paths["calib"])
lidar_points = load_lidar_bin(paths["lidar"])
lidar_points_cam = filter_lidar_fov(lidar_points, fov_degrees=80)
lidar_points_cam = remove_ground_plane(lidar_points_cam, height_threshold=-1.5, axis='z')
lidar_points_cam = transform_lidar_to_camera(lidar_points_cam, Tr_velo_to_cam)

# ---------- K3D Visualization ---------- #
plot = k3d.plot(camera_auto_fit=False, axes_helper=0.0)

# Plot LIDAR points in blue
lidar_points_cam_rect = (R0_rect @ lidar_points_cam.T).T
plot += k3d.points(lidar_points_cam_rect, point_size=0.05, color=0x0000ff)      # Plot rectified LiDAR Pointcloud in green
#     plot += k3d.points(lidar_points_cam, point_size=0.05, color=0x00ff00)           # Plot non-rectified Lidar Pointcloud in blue


# Load left image and disparity
# left_img = cv2.imread(paths["left_img"], cv2.IMREAD_COLOR)
# disparity = cv2.imread(paths["disparity"], cv2.IMREAD_GRAYSCALE).astype(np.float32)

# xyz_img = compute_3D_from_disparity(P3, disparity)                             # Not rectified, add R0_rect to rectify it

## Downsample disparity maps --> only get the forth pixel for example
# Subsample disparity map before computing 3D
# stride = 4  # every 4th pixel → 16x fewer points

# Downsample disparity and image using slicing
# disparity_sub = disparity[::stride, ::stride]
# left_img_sub = left_img[::stride, ::stride]

# Recompute point cloud on the smaller disparity map
# xyz_img_sub = compute_3D_from_disparity(P3, disparity_sub, R0_rect)

# disp_points = xyz_img.reshape(-1, 3)
# disp_points = disp_points[~np.isnan(disp_points).any(axis=1)]

# Remove ground and sky of disparity point cloud
# disp_points = remove_ground_plane(disp_points, height_threshold=1.3, axis='y', direction='below')
# disp_points = remove_ground_plane(disp_points, height_threshold=-1.0, axis='y', direction='above')

# Create colors as integers for valid points
# valid_mask = ~np.isnan(xyz_img[:, :, 0])
# img_colors = left_img[valid_mask]
# img_colors = img_colors[:disp_points.shape[0]]
# color_ints = bgr_to_int(img_colors)


# Plot using K3D
#     plot += k3d.points(positions=disp_points, colors=color_ints, point_size=0.025)
# plot += k3d.points(positions=disp_points, point_size=0.025, colors=color_ints)



labels_df = extract_labels(paths["labels"])
pred_lidar_df = extract_predictions(paths["pred_lidar"])

filtered_labels = filter_labels_in_fov(labels_df, R0_rect, fov_degrees=80)
filtered_pred_lidar = filter_labels_in_fov(pred_lidar_df, R0_rect, fov_degrees=80)

draw_3d_bounding_boxes_k3d_with_labels(plot, filtered_labels, R0_rect, color=0xff0000)
draw_3d_bounding_boxes_k3d_with_labels(plot, filtered_pred_lidar, R0_rect, color=0x00ff00)


print(filtered_pred_lidar)




plot.camera = [0, 0, 0, 0, 0, 15, 0, -1.0, 0]
plot.display()

Frame: 924
   ObjectType  Truncation  Occlusion     Alpha           X1          Y1  \
0  Pedestrian         0.0        0.0  1.028235   418.259003  174.063751   
1  Pedestrian         0.0        0.0 -1.068551   803.044800  183.419037   
2  Pedestrian         0.0        0.0  1.962524  1082.158813  178.536591   
3  Pedestrian         0.0        0.0 -1.234682  1134.553467  195.273392   
4     Cyclist         0.0        0.0 -1.524096   686.755310  162.804474   
5     Cyclist         0.0        0.0 -1.997118   976.407104  125.850807   
6     Cyclist         0.0        0.0 -1.862206   786.330566  175.904388   
7         Car         0.0        0.0 -0.985404    25.055443   88.458984   
8         Car         0.0        0.0  1.463253   765.929993  176.641907   
9         Car         0.0        0.0 -0.659768    11.504411  192.223877   

            X2          Y2         L         H         W          X         Y  \
0   473.787201  262.799316  0.847244  1.732670  0.774069  -3.387983  1.758599   
1

/home/simina/miniconda3/envs/cv_env/lib/python3.10/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()